# ⚙️ Week 09-10 · Notebook 03 · Tensors & GPU Acceleration in Edge Plants

Benchmark tensor operations and GPU utilization strategies when compute is limited to shared workstations or on-prem clusters.

## 🎯 Learning Objectives
- Manipulate PyTorch tensors across CPU/GPU devices.
- Evaluate mixed-precision trade-offs for manufacturing workloads.
- Profile memory and throughput to align with plant uptime schedules.
- Capture governance notes for IT change management.

## 🧩 Scenario
A tier-2 supplier alternates between an A100 workstation and a shared RTX 6000. Training jobs must finish overnight without impacting SCADA traffic. You need a repeatable benchmark harness and downtime mitigation plan.

In [ ]:
import torch
import time
import pandas as pd

torch.manual_seed(1234)

## 🧮 Tensor Fundamentals
Inspect dtype, device, and stride configuration to confirm compliance with IT policies.

In [ ]:
vector = torch.arange(12, dtype=torch.float32).reshape(3, 4)
vector
vector.stride(), vector.device

If a GPU is available, move tensors and compare memory footprints. Fallback gracefully when running on CPU-only plant laptops.

In [ ]:
def tensor_device_summary():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    t = torch.rand((2048, 2048), device=device, dtype=torch.float32)
    summary = {
    return summary

tensor_device_summary()

## ⚡ Mixed Precision Benchmark
Evaluate float32 vs. bfloat16/float16 throughput. Use caution on safety-critical inference pipelines.

In [ ]:
def matmul_benchmark(size=4096, dtype=torch.float32, device='cpu', runs=5):
    a = torch.randn((size, size), dtype=dtype, device=device)
    b = torch.randn((size, size), dtype=dtype, device=device)
    torch.cuda.synchronize() if device == 'cuda' else None
    times = []
    for _ in range(runs):
    return sum(times) / len(times)

def benchmark_suite():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtypes = [torch.float32]
    if device == 'cuda':
    results = []
    for dtype in dtypes:
    return pd.DataFrame(results)

benchmark_suite()

## 🧾 Memory Profiling Checklist
- Capture `torch.cuda.memory_summary()` at job start and end.
- Enforce IT's GPU allocation window (e.g., 18:00-06:00) to avoid SCADA conflicts.
- Log utilization metrics in maintenance CMMS for accountability.

In [ ]:
if torch.cuda.is_available():
    summary_text = torch.cuda.memory_summary(device=torch.device('cuda'), abbreviated=True)
    summary_text.split('
')[:10]
else:

## 🛡️ Downtime Mitigation Plan
| Risk | Mitigation | Owner |

## 🧪 Lab Assignment
1. Run the benchmark suite on both A100 and RTX 6000, compare throughput.
2. Add power draw instrumentation (e.g., `nvidia-smi --query-gpu=power.draw`).
3. Propose a mixed-precision policy for safety-critical vs. internal tools.
4. Submit change-management ticket with benchmark evidence and rollback plan.

## ✅ Checklist
- [ ] Tensor device audit completed
- [ ] Mixed-precision benchmark logged
- [ ] Memory summary archived for compliance
- [ ] Downtime mitigation plan approved

## 📚 References
- PyTorch Performance Tuning Guide
- NVIDIA A100 vs. RTX 6000 Comparison (2025)
- *Operational Technology Change Management Handbook* (ISA, 2023)